### Import Libraries

In [1]:
import pandas as pd
import random
import os
import numpy as np
from functools import partial
from lightgbm import LGBMRegressor
from hyperopt import fmin, hp, tpe, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [3]:
def dataset_split_X_y(df):    
    """
    @Description: split data into features and labels
    @Param: df, pandas dataframe with columns starting with X for features and Y for labels
    @Return: features and labels in pandas dataframes
    """
    xs = df.filter(regex='X') # Input : X Feature
    ys = df.filter(regex='Y') # Output : Y Feature
    return xs, ys

In [4]:
def zero_variance(df):
    """
    @Description: check for zero_variance
    @Param1: df, pandas dataframe
    @Return: names of the columns with zero variance
    """
    result = []
    for col in df.columns:
        if df[col].var() == 0:
            result.append(col)
    return result

In [5]:
def get_top_correlation(df, n=10):
    """
    @Description: print out top correlated features
    @Param1: df, pandas dataframe
    @Param2: n, number of lines to print 
    @Return: pandas series
    """
    pairs = set()
    for idx1 in range(0, df.shape[1]):
        for idx2 in range(0, idx1+1):
            pairs.add((df.columns[idx1], df.columns[idx2]))
    corr = df.corr().abs().unstack()
    corr = corr.drop(labels=pairs).sort_values(ascending=False)
    return corr[0:n]

In [6]:
def lg_nrmse(gt, preds):
    """
    @Description: Metric used in this project
    @Params1: gt, pandas dataframe
    @Param2: preds, pandas dataframe
    @Return: nrmse score
    """
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    preds = pd.DataFrame(preds)
    all_nrmse = []
    for idx in range(0,14):
        rmse = mean_squared_error(gt.iloc[:,idx], preds.iloc[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [51]:
train_x = pd.read_csv('data/train.csv')

test_x = pd.read_csv('data/test.csv')
train_x, train_y = dataset_split_X_y(train_x)


y_feature_spec_info = pd.read_csv('data/meta/y_feature_spec_info.csv')

In [8]:
train_y.head()

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,2.056,1.456,1.680,10.502,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,1.446,1.184,1.268,18.507,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,1.251,0.665,0.782,14.082,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,1.464,1.079,1.052,16.975,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,0.983,0.646,0.689,15.047,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974


In [52]:
train_spec_df = y_feature_spec_info
df_indicator_y = pd.DataFrame()
for idx in range(len(train_spec_df.Feature)):
    if train_spec_df.Feature[idx] in train_y.columns:
        y_series = ~train_y[train_spec_df.Feature[idx]].between(train_spec_df.iloc[idx, :].Min, train_spec_df.iloc[idx, :].Max)
        df_indicator_y = pd.concat([df_indicator_y, y_series.astype(int)], axis = 1)

In [53]:
normal_data = df_indicator_y[df_indicator_y==0]
spec_data = df_indicator_y[df_indicator_y==1]

In [54]:
spec_data.count()

Y_01    1476
Y_02     558
Y_03     464
Y_04     500
Y_05      91
Y_06      10
Y_07    1822
Y_08      19
Y_09      19
Y_10       5
Y_11       3
Y_12      16
Y_13      15
Y_14      13
dtype: int64

In [55]:
train_x_spec_y_01 = train_x[spec_data['Y_02'] == 1]
train_x_normal_y_01 = train_x[normal_data['Y_02'] == 0]

In [56]:
train_x_spec_y_01['X_01']

23       64.425
37       74.623
177      77.682
239      69.524
246      67.485
          ...  
39352    66.465
39378    68.504
39393    68.504
39583    67.485
39605    66.465
Name: X_01, Length: 558, dtype: float64

In [57]:
for k in train_y.columns:
    train_x_spec_y_01 = train_x[spec_data[k] == 1]
    train_x_normal_y_01 = train_x[normal_data[k] == 0]
    print(k)
    for i in train_x.columns:
        if max(train_x_normal_y_01[i]) < max(train_x_spec_y_01[i]):
            print('max :' ,i)
        if min(train_x_normal_y_01[i]) > min(train_x_spec_y_01[i]) :
            print('min :' ,i)
    print()




Y_01
min : X_14
min : X_15
min : X_16
min : X_18
max : X_51

Y_02
min : X_49

Y_03
min : X_49

Y_04
max : X_37
max : X_54

Y_05

Y_06

Y_07
min : X_41

Y_08

Y_09

Y_10

Y_11

Y_12

Y_13

Y_14



In [58]:
train_x_normal_y_01.iloc[:,10:20].describe()

,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20
count,39594.000000,39594.000000,39594.000000,39594.000000,39594.000000,39594.000000,39594.000000,39594.000000,39594.000000,39594.000000
mean,0.000366,4.373230,0.143335,13.372203,13.381916,13.463860,13.512590,13.449264,3.240250,3.184512
std,0.014148,0.021808,0.025332,0.029867,0.029468,0.036741,0.023436,0.029094,0.110484,0.105266
min,0.000000,4.270000,0.050000,13.150000,13.230000,13.260000,13.410000,13.260000,2.860000,2.830000
25%,0.000000,4.360000,0.130000,13.350000,13.360000,13.440000,13.500000,13.430000,3.160000,3.100000
50%,0.000000,4.370000,0.140000,13.370000,13.380000,13.470000,13.510000,13.450000,3.220000,3.180000
75%,0.000000,4.390000,0.160000,13.390000,13.410000,13.490000,13.530000,13.470000,3.310000,3.270000
max,0.700000,4.490000,0.280000,13.490000,13.500000,13.610000,13.610000,13.570000,3.750000,3.670000


In [66]:
train_x_spec_y_01.iloc[:,10:20].describe()

,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20
count,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000
mean,0.001075,4.372688,0.145860,13.370358,13.382348,13.461649,13.513799,13.449713,3.268602,3.215072
std,0.025400,0.020486,0.025122,0.029096,0.029234,0.038131,0.023544,0.028219,0.131336,0.110240
min,0.000000,4.310000,0.070000,13.270000,13.260000,13.350000,13.430000,13.350000,2.910000,2.910000
25%,0.000000,4.360000,0.130000,13.350000,13.360000,13.440000,13.500000,13.430000,3.170000,3.120000
50%,0.000000,4.370000,0.140000,13.370000,13.380000,13.460000,13.520000,13.450000,3.250000,3.230000
75%,0.000000,4.380000,0.160000,13.390000,13.400000,13.490000,13.530000,13.460000,3.380000,3.310000
max,0.600000,4.440000,0.250000,13.460000,13.450000,13.560000,13.570000,13.530000,3.600000,3.450000


In [67]:
train_x_normal_y_01.iloc[:,20:30].describe()

,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30
count,39049.000000,39049.000000,39049.0,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000
mean,3.173867,3.232196,1.0,2.115659,2.093904,2.090331,2.098221,2.118552,2.173698,1.379079
std,0.106594,0.108809,0.0,0.032451,0.033155,0.038500,0.038003,0.042768,0.046691,0.029953
min,2.830000,2.850000,1.0,1.830000,1.960000,1.980000,1.990000,1.930000,2.020000,0.570000
25%,3.090000,3.140000,1.0,2.090000,2.070000,2.060000,2.070000,2.090000,2.140000,1.370000
50%,3.160000,3.230000,1.0,2.120000,2.090000,2.090000,2.090000,2.120000,2.170000,1.370000
75%,3.250000,3.320000,1.0,2.140000,2.120000,2.120000,2.120000,2.140000,2.200000,1.380000
max,3.680000,3.790000,1.0,2.350000,2.350000,2.350000,2.350000,2.350000,2.360000,2.110000


In [68]:
train_x_spec_y_01.iloc[:,20:30].describe()

,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30
count,558.000000,558.000000,558.0,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000
mean,3.202491,3.266039,1.0,2.116720,2.093244,2.093602,2.100197,2.121541,2.175753,1.373065
std,0.121381,0.115769,0.0,0.031665,0.032717,0.039304,0.041560,0.043679,0.046513,0.053234
min,2.880000,3.000000,1.0,2.030000,2.010000,2.010000,2.010000,2.040000,2.090000,0.570000
25%,3.100000,3.180000,1.0,2.090000,2.070000,2.070000,2.070000,2.090000,2.140000,1.370000
50%,3.185000,3.280000,1.0,2.120000,2.090000,2.090000,2.100000,2.120000,2.170000,1.370000
75%,3.300000,3.360000,1.0,2.140000,2.120000,2.120000,2.120000,2.140000,2.200000,1.380000
max,3.550000,3.600000,1.0,2.190000,2.210000,2.320000,2.350000,2.350000,2.340000,1.510000


In [69]:
train_x_normal_y_01.iloc[:,30:40].describe()

,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40
count,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000
mean,1.571179,1.362961,1.595846,12.950250,12.920309,12.941730,12.919113,-15.904671,-15.890222,-16.572931
std,0.074738,0.030060,0.107967,0.044023,0.052232,0.047839,0.052295,0.594186,0.747688,0.343985
min,0.600000,0.570000,0.610000,12.840000,12.810000,12.840000,12.810000,-17.090000,-17.090000,-17.720000
25%,1.530000,1.350000,1.550000,12.920000,12.870000,12.900000,12.870000,-16.160000,-16.160000,-16.810000
50%,1.550000,1.360000,1.570000,12.960000,12.910000,12.950000,12.910000,-15.990000,-15.990000,-16.640000
75%,1.600000,1.370000,1.610000,12.990000,12.970000,12.980000,12.970000,-15.750000,-15.750000,-16.400000
max,7.210000,2.450000,7.810000,13.080000,13.090000,13.090000,13.080000,32.230000,-2.650000,-14.800000


In [70]:
train_x_spec_y_01.iloc[:,30:40].describe()

,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40
count,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000
mean,1.568495,1.360663,1.587151,12.951685,12.922724,12.941631,12.920914,-15.838423,-15.844373,-16.518315
std,0.057227,0.020243,0.112019,0.044521,0.052634,0.047217,0.052147,0.673639,0.670276,0.374343
min,1.500000,1.330000,0.610000,12.850000,12.830000,12.850000,12.810000,-16.760000,-16.760000,-17.360000
25%,1.530000,1.350000,1.550000,12.920000,12.870000,12.900000,12.880000,-16.140000,-16.140000,-16.790000
50%,1.550000,1.360000,1.570000,12.960000,12.925000,12.950000,12.920000,-15.940000,-15.960000,-16.605000
75%,1.590000,1.370000,1.590000,12.990000,12.970000,12.980000,12.970000,-15.640000,-15.650000,-16.290000
max,1.880000,1.470000,2.160000,13.040000,13.060000,13.040000,13.030000,-2.650000,-2.650000,-15.340000


In [71]:
train_x_normal_y_01.iloc[:,40:50].describe()

,X_41,X_42,X_43,X_44,X_45,X_46,X_47,X_48,X_49,X_50
count,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.0,39049.0,39049.000000,39049.000000
mean,21.186992,21.059368,21.203696,21.160097,0.154520,1468.276166,1.0,1.0,16678.583539,130.782024
std,0.031099,0.040272,0.047227,0.042166,0.046989,2.120836,0.0,0.0,8552.008319,5.991171
min,20.730000,20.790000,20.800000,20.930000,0.000000,1457.000000,1.0,1.0,3382.630000,21.800000
25%,21.170000,21.030000,21.170000,21.130000,0.120000,1469.000000,1.0,1.0,13108.630000,126.956647
50%,21.190000,21.060000,21.200000,21.160000,0.150000,1469.000000,1.0,1.0,15278.930000,130.732472
75%,21.210000,21.090000,21.240000,21.190000,0.190000,1469.000000,1.0,1.0,17564.730000,134.547360
max,21.620000,21.440000,21.410000,21.320000,0.420000,1469.000000,1.0,1.0,114563.630000,162.619458


In [72]:
train_x_spec_y_01.iloc[:,40:50].describe()

,X_41,X_42,X_43,X_44,X_45,X_46,X_47,X_48,X_49,X_50
count,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.0,558.0,558.000000,558.000000
mean,21.186953,21.056971,21.204337,21.160771,0.157849,1468.215054,1.0,1.0,16560.053513,130.252055
std,0.033132,0.041350,0.046088,0.042851,0.045367,2.169725,0.0,0.0,10618.710723,5.878046
min,21.080000,20.890000,21.040000,21.000000,0.040000,1457.000000,1.0,1.0,3341.830000,113.547627
25%,21.170000,21.030000,21.170000,21.130000,0.120000,1469.000000,1.0,1.0,12175.205000,126.111388
50%,21.190000,21.060000,21.210000,21.160000,0.160000,1469.000000,1.0,1.0,14783.880000,130.313973
75%,21.210000,21.080000,21.230000,21.200000,0.190000,1469.000000,1.0,1.0,17264.080000,133.887299
max,21.510000,21.250000,21.370000,21.240000,0.290000,1469.000000,1.0,1.0,114211.130000,150.277661


In [73]:
train_x_normal_y_01.describe()

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
count,39049.000000,39049.000000,39049.000000,39049.0,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,...,39049.0,39049.0,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000,39049.000000
mean,68.411610,103.320167,68.816570,1.0,102.337957,70.595443,29.394517,164.448757,225.514783,0.002407,...,1.0,1.0,16678.583539,130.782024,131.459133,138.587340,127.995251,128.017585,137.886895,128.446802
std,2.658364,0.000373,5.154366,0.0,0.548553,2.260763,7.258563,220.629270,66.638349,0.085533,...,0.0,0.0,8552.008319,5.991171,5.941484,6.474068,5.712359,5.441066,6.558856,5.450783
min,56.268000,103.320000,56.470000,1.0,101.774000,61.726000,14.140000,38.460000,37.580000,0.000000,...,1.0,1.0,3382.630000,21.800000,21.910000,23.100000,21.330000,21.340000,22.980000,21.410000
25%,66.465000,103.320000,65.070000,1.0,101.949000,68.864000,27.890000,105.960000,188.630000,0.000000,...,1.0,1.0,13108.630000,126.956647,127.668461,134.471696,124.379934,124.696049,133.751523,125.140644
50%,68.504000,103.320000,67.270000,1.0,102.006000,69.884000,28.840000,115.040000,234.580000,0.000000,...,1.0,1.0,15278.930000,130.732472,131.356266,138.524881,128.027021,128.103878,137.901829,128.415753
75%,69.524000,103.320000,71.770000,1.0,103.144000,71.923000,29.870000,132.630000,263.960000,0.000000,...,1.0,1.0,17564.730000,134.547360,135.173489,142.704385,131.618765,131.499576,142.075490,131.842556
max,84.820000,103.321000,89.170000,1.0,103.160000,87.219000,163.860000,2387.440000,637.490000,3.600000,...,1.0,1.0,114563.630000,162.619458,194.513195,173.438623,152.406630,175.052891,170.155980,155.277538


In [74]:
train_x_spec_y_01.describe()

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
count,558.000000,558.000000,558.000000,558.0,558.000000,558.00000,558.000000,558.000000,558.000000,558.000000,...,558.0,558.0,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000,558.000000
mean,68.442138,103.320149,69.511039,1.0,102.284437,70.72097,30.315323,164.488692,217.187832,0.005376,...,1.0,1.0,16560.053513,130.252055,131.337355,138.605408,127.794313,127.770899,137.731848,128.043417
std,2.485748,0.000356,4.877934,0.0,0.531973,2.19123,11.601906,204.086726,72.760905,0.127000,...,0.0,0.0,10618.710723,5.878046,5.412467,6.170294,5.770202,4.909347,6.064247,4.953707
min,58.307000,103.320000,60.270000,1.0,101.815000,63.76600,24.960000,58.380000,37.580000,0.000000,...,1.0,1.0,3341.830000,113.547627,108.960072,114.213213,109.122007,113.056164,120.323552,101.781635
25%,66.465000,103.320000,65.770000,1.0,101.932250,68.86400,28.192500,107.277500,176.275000,0.000000,...,1.0,1.0,12175.205000,126.111388,127.694327,134.810781,123.699215,124.752235,133.336472,124.771286
50%,68.504000,103.320000,68.870000,1.0,101.998000,70.90400,28.885000,115.565000,226.360000,0.000000,...,1.0,1.0,14783.880000,130.313973,131.157730,138.598226,127.528109,127.865219,137.789589,128.311512
75%,69.524000,103.320000,72.645000,1.0,103.136750,71.92300,29.967500,128.570000,264.917500,0.000000,...,1.0,1.0,17264.080000,133.887299,134.592182,142.458873,131.542738,130.825727,141.862725,131.575552
max,77.682000,103.321000,84.070000,1.0,103.160000,78.04200,163.860000,2358.070000,619.900000,3.000000,...,1.0,1.0,114211.130000,150.277661,147.817028,158.631830,145.078482,144.995846,155.575505,140.942455


In [59]:
def outlier_iqr_upper(tmp, i):
    data = tmp[i]
    
    print(np.percentile(data,75))
    print(np.percentile(data,25))
    q25, q75 = np.percentile(data, 25), np.percentile(data,75)
    iqr = q75 - q25
    
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    print('변수 명 : ',i)
    print('IQR : ', iqr)
    print('lower bound : ', lower)
    print('upper bound : ', upper)
    print(np.where(train_x['X_57'] == 1, 1, np.where(data>upper, 1, 0)))
    
    train_x['X_57'] = np.where(train_x['X_57'] == 1, 1, np.where(data>upper, 1, 0))
    
    print("tmp['X_57'].value_counts() : ", train_x['X_57'].value_counts());
    return tmp

In [60]:
def outlier_iqr_lower(tmp, i):
    data = tmp[i]
    
    print(np.percentile(data,75))
    print(np.percentile(data,25))
    q25, q75 = np.percentile(data, 25), np.percentile(data,75)
    iqr = q75 - q25
    
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    print('변수 명 : ',i)
    print('IQR : ', iqr)
    print('lower bound : ', lower)
    print('upper bound : ', upper)
    train_x['X_57'] = np.where(train_x['X_57'] == 1, 1, np.where(data<lower, 1, 0))
    
    print("tmp['X_57'].value_counts() : ", train_x['X_57'].value_counts());
    return tmp

In [61]:
X_57 = [0 for i in range(0, train_x.shape[0])]
len(X_57)
X_57 = pd.DataFrame(X_57)
train_x['X_57'] = X_57

In [62]:
# 2차 시도 (y와 corr 기준) fixed 2 !!!!!!!!!!
cols_lower = ["X_14","X_15","X_16", "X_18","X_41", "X_49"]
cols_upper = ["X_51",  "X_37", "X_54"]


In [18]:
# 2차 시도 (y와 corr 기준) fixed 2 !!!!!!!!!!
cols = ["X_14","X_15", "X_17", "X_18", "X_19", "X_20", "X_21","X_22",
        "X_24", "X_25", "X_26", "X_27", "X_28","X_29", "X_39", "X_40","X_41","X_42","X_43", "X_44","X_45"]


In [19]:
cols = ["X_11", "X_14","X_15", "X_16", "X_17", "X_18", "X_19", "X_20", "X_21","X_22",
        "X_24", "X_25", "X_26", "X_27", "X_28","X_29", 
        "X_30", "X_31", "X_32", "X_33",
        "X_34", "X_35", "X_36", "X_37", 
        "X_39", "X_40","X_41","X_42","X_43", "X_44","X_45"
       ]

In [63]:
for i in cols_upper:
    q25, q75 = np.percentile(train_x[i], 25), np.percentile(train_x[i],75)
    iqr = q75 - q25
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    train_x['X_57'] = np.where(train_x['X_57'] == 1, 1, np.where(train_x[i]>upper, 1, 0))

    

In [64]:
for i in cols_lower:
    q25, q75 = np.percentile(train_x[i], 25), np.percentile(train_x[i],75)
    iqr = q75 - q25
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    train_x['X_57'] = np.where(train_x['X_57'] == 1, 1, np.where(train_x[i]<lower, 1, 0))
    

In [65]:
train_x['X_57'].value_counts()

0    38357
1     1250
Name: X_57, dtype: int64

In [66]:

X_57 = [0 for i in range(0, test_x.shape[0])]
X_57 = pd.DataFrame(X_57)
test_x['X_57'] = X_57

for i in cols_upper:
    q25, q75 = np.percentile(test_x[i], 25), np.percentile(test_x[i],75)
    iqr = q75 - q25
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    test_x['X_57'] = np.where(test_x['X_57'] == 1, 1, np.where(test_x[i]>upper, 1, 0))

    

In [67]:
for i in cols_lower:
    q25, q75 = np.percentile(test_x[i], 25), np.percentile(test_x[i],75)
    iqr = q75 - q25
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    test_x['X_57'] = np.where(test_x['X_57'] == 1, 1, np.where(test_x[i]<lower, 1, 0))
    

In [46]:

cols_with_zero_variance = zero_variance(train_x)
train_x = train_x.drop(cols_with_zero_variance, axis = 1)
test_x = test_x.drop(cols_with_zero_variance, axis = 1)

highly_correlated = [i[1] for i in get_top_correlation(train_x, 3).index]
#train_x = train_x.drop(highly_correlated, axis = 1)

#test_x = test_x.drop(highly_correlated, axis = 1)
#test_x = test_x.drop('ID', axis=1)

In [68]:
def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'num_leaves': int(params['num_leaves']),
        'min_child_samples': int(params['min_child_samples']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'subsample': '{:.3f}'.format(params['subsample']),
        'min_split_gain': '{:.3f}'.format(params['min_split_gain']),
        'scale_pos_weight': '{:.3f}'.format(params['scale_pos_weight']),
        'reg_alpha': '{:.3f}'.format(params['reg_alpha']),
        'reg_lambda': '{:.3f}'.format(params['reg_lambda']),
        'learning_rate': '{:.3f}'.format(params['learning_rate']),
        
    }
    
    model = MultiOutputRegressor(LGBMRegressor(n_jobs = -1, random_state = 1, **params))
    
    loss = -cross_val_score(model, train_x, train_y, cv=10, scoring=make_scorer(lg_nrmse, greater_is_better=False)).mean()
    print("NRMSE Loss {:.5f} params {}".format(loss, params))
    return loss

In [69]:
space = {
    'n_estimators' : hp.quniform('n_estimators', 100, 1500, 58),
    'max_depth': hp.quniform('max_depth', 3, 100, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 100, 10),
    'min_child_samples': hp.quniform('min_child_samples', 10, 300, 10),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'subsample': hp.uniform('subsample', 0.3, 1.0),
    'min_split_gain': hp.uniform('min_split_gain', 0, 0.7),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 1, 10),
    'reg_alpha': hp.uniform('reg_alpha', 0, 100),
    'reg_lambda': hp.uniform('reg_lambda', 0, 100),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
}

best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 200,
            rstate=np.random.default_rng(1))

NRMSE Loss 1.94459 params {'n_estimators': 1334, 'max_depth': 47, 'num_leaves': 20, 'min_child_samples': 240, 'colsample_bytree': '0.715', 'subsample': '0.657', 'min_split_gain': '0.655', 'scale_pos_weight': '4.117', 'reg_alpha': '22.649', 'reg_lambda': '55.509', 'learning_rate': '0.042'}
NRMSE Loss 1.94481 params {'n_estimators': 1508, 'max_depth': 13, 'num_leaves': 90, 'min_child_samples': 110, 'colsample_bytree': '0.538', 'subsample': '0.901', 'min_split_gain': '0.652', 'scale_pos_weight': '7.202', 'reg_alpha': '1.693', 'reg_lambda': '75.762', 'learning_rate': '0.159'}
NRMSE Loss 1.95297 params {'n_estimators': 232, 'max_depth': 67, 'num_leaves': 80, 'min_child_samples': 170, 'colsample_bytree': '0.387', 'subsample': '0.902', 'min_split_gain': '0.436', 'scale_pos_weight': '8.879', 'reg_alpha': '86.379', 'reg_lambda': '88.854', 'learning_rate': '0.039'}
NRMSE Loss 1.95513 params {'n_estimators': 580, 'max_depth': 27, 'num_leaves': 60, 'min_child_samples': 120, 'colsample_bytree': '0.

KeyboardInterrupt: 

In [ ]:
model = MultiOutputRegressor(LGBMRegressor(n_jobs = -1, random_state = 1, **best))
model.fit(train_x, train_y)
preds = model.predict(test_x)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
submit.to_csv('./submission_3.csv', index = False)